In [1]:
import os
import sys
import importlib as imp
import cooler
import shelve
from collections import defaultdict

from pandas.api.types import CategoricalDtype

from cooler import Cooler

import mitosis_manager as mm
%matplotlib inline

# Creating initial database from list of cooler files

### Processing cooler library names

In [2]:
libraries = []
for filename in np.sort(os.listdir(f'{mm.base_path}/coolers/')):
    print(filename)
    if 'mapq_30.1000.mcool' in filename:
        f = filename.split('.g')[0].split('__')[0]
        libraries.append(f)
        
libraries = np.sort(np.asarray(libraries))
# libraries

AID-KIF4-10min-S3-R3-pna__galGal7b.mapq_30.1000.mcool
AID-KIF4-15min-S3-R3-pna__galGal7b.mapq_30.1000.mcool
AID-KIF4-2.5min-S3-R1-pna__galGal7b.mapq_30.1000.mcool
AID-KIF4-30min-S3-R3-pna__galGal7b.mapq_30.1000.mcool
AID-KIF4-5min-S3-R1-pna__galGal7b.mapq_30.1000.mcool
AID-KIF4-7.5min-S3-R3-pna__galGal7b.mapq_30.1000.mcool
AID-KIF4-G2-S3-R5-pna__galGal7b.mapq_30.1000.mcool
AID-KIF4-G2-S3-R6-pna__galGal7b.mapq_30.1000.mcool
CAPH-AID-10min-S3-R3-spnA__galGal7b.mapq_30.1000.mcool
CAPH-AID-15min-S3-R3-spnA__galGal7b.mapq_30.1000.mcool
CAPH-AID-2.5min-S3-R3-spnA__galGal7b.mapq_30.1000.mcool
CAPH-AID-30min-S3-R3-spnA__galGal7b.mapq_30.1000.mcool
CAPH-AID-5min-S3-R3-spnA__galGal7b.mapq_30.1000.mcool
CAPH-AID-7.5min-S3-R3-spnA__galGal7b.mapq_30.1000.mcool
CAPH-AID-G2-S3-R5-spnA__galGal7b.mapq_30.1000.mcool
CAPH2-AID-10min-S3-R3-spnA__galGal7b.mapq_30.1000.mcool
CAPH2-AID-10min-S4-R4-spnA__galGal7b.mapq_30.1000.mcool
CAPH2-AID-10min-S5-R5-spNA__galGal7b.mapq_30.1000.mcool
CAPH2-AID-10min-S6-R6-

In [3]:
libname = []
condition = []
timepoint = []
replicate = []
is_sorted = []
noc = []
aux = []

isupper = lambda s: s == s.upper()

for item in libraries:
    if item == 'No_SMC3' or item == 'SMC3':
        continue
#     print(item)

    libname.append(item)

    item = item.split('-')
    if len(item[-1]) == 5:        
        cond = '-'.join(item[0:-6])
        time = item[-5]
        batch = item[-4]
    else:
        cond = '-'.join(item[0:-5])
        time = item[-4]
        batch = item[-3]

    if cond == 'AID':
        cond = 'KIF4'
    if cond == '':
        cond = 'WT'
    
#     print('\t', cond, time, batch)


    condition.append(cond)
    timepoint.append(time)
    replicate.append(batch)
    
    treatment = item[-1]
    if len(treatment) == 3:
        s = np.nan
#         p = False
        n = False
        a = False
        
    elif len(treatment) == 5:
        s = True
#         p = True
        n = False
        a = treatment[-2:]
        
    else:
        s = isupper(treatment[0])
#         p = isupper(treatment[2])
        n = isupper(treatment[2])
        a = isupper(treatment[3])
    
    is_sorted.append(s)
#     palbo.append(p)
    noc.append(n)
    aux.append(a)


df = pd.DataFrame({'lib_name':libname,
                   'condition':condition,
                   'time':timepoint,
                   'replicate':replicate,
                   'S':is_sorted,
                   'N/M':noc,
                   'A':aux
                  })

df.loc[(df['condition']=='WAPL') & (df['time']=='11min'),'time'] = '10min'

cat_type = CategoricalDtype(categories=['G2', '2.5min', '5min','7.5min',
                                        '10min','15min','30min','45min',
                                        '60min', '75min','90min'], ordered=True
                           )
df['time'] = df['time'].astype(cat_type)
df = df.sort_values(['condition','replicate','time']).reset_index(drop=True)


caph_mask = (df['condition']=='CAPH')
t5_mask = caph_mask & (df['time']=='5min')
t75_mask = caph_mask & (df['time']=='7.5min')
df.loc[t5_mask,'time'] = '7.5min'
df.loc[t75_mask,'time'] = '5min'


smc2_mask = (df['condition']=='SMC2') & (df['replicate']=='S4')
t75_mask = smc2_mask & (df['time']=='7.5min')
df.drop(df.loc[t75_mask].index.values, inplace=True)

t10_mask = smc2_mask & (df['time']=='10min')
t15_mask = smc2_mask & (df['time']=='15min')
df.loc[t15_mask,'time'] = '10min'
df.loc[t10_mask,'time'] = '15min'



smc3caph2s2_mask = (df['condition']=='SMC3-CAPH2') & (df['replicate']=='S2')
df.loc[smc3caph2s2_mask, 'time'] = df.loc[smc3caph2s2_mask, 'time'].values[::-1]

df = df.sort_values(['condition','replicate','time']).reset_index(drop=True)

for _, group in df.groupby(['condition','replicate']):
    display(group)

,lib_name,condition,time,replicate,S,N/M,A
0,CAPH-AID-G2-S3-R5-spnA,CAPH,G2,S3,False,False,True
1,CAPH-AID-2.5min-S3-R3-spnA,CAPH,2.5min,S3,False,False,True
2,CAPH-AID-7.5min-S3-R3-spnA,CAPH,5min,S3,False,False,True
3,CAPH-AID-5min-S3-R3-spnA,CAPH,7.5min,S3,False,False,True
4,CAPH-AID-10min-S3-R3-spnA,CAPH,10min,S3,False,False,True
5,CAPH-AID-15min-S3-R3-spnA,CAPH,15min,S3,False,False,True
6,CAPH-AID-30min-S3-R3-spnA,CAPH,30min,S3,False,False,True


,lib_name,condition,time,replicate,S,N/M,A
7,CAPH2-AID-G2-S3-R5-spnA,CAPH2,G2,S3,False,False,True
8,CAPH2-AID-2.5min-S3-R3-spnA,CAPH2,2.5min,S3,False,False,True
9,CAPH2-AID-5min-S3-R3-spnA,CAPH2,5min,S3,False,False,True
10,CAPH2-AID-7.5min-S3-R3-spnA,CAPH2,7.5min,S3,False,False,True
11,CAPH2-AID-10min-S3-R3-spnA,CAPH2,10min,S3,False,False,True
12,CAPH2-AID-15min-S3-R3-spnA,CAPH2,15min,S3,False,False,True
13,CAPH2-AID-30min-S3-R3-spnA,CAPH2,30min,S3,False,False,True


,lib_name,condition,time,replicate,S,N/M,A
14,CAPH2-AID-G2-S4-R6-spnA,CAPH2,G2,S4,False,False,True
15,CAPH2-AID-5min-S4-R4-spnA,CAPH2,5min,S4,False,False,True
16,CAPH2-AID-10min-S4-R4-spnA,CAPH2,10min,S4,False,False,True
17,CAPH2-AID-15min-S4-R4-spnA,CAPH2,15min,S4,False,False,True
18,CAPH2-AID-15min-S4-R5-spNA,CAPH2,15min,S4,False,True,True
19,CAPH2-AID-15min-S4-R6-spNA,CAPH2,15min,S4,False,True,True
20,CAPH2-AID-30min-S4-R4-spNa,CAPH2,30min,S4,False,True,False
21,CAPH2-AID-30min-S4-R4-spnA,CAPH2,30min,S4,False,False,True
22,CAPH2-AID-30min-S4-R5-spNA,CAPH2,30min,S4,False,True,True
23,CAPH2-AID-30min-S4-R6-spNA,CAPH2,30min,S4,False,True,True


,lib_name,condition,time,replicate,S,N/M,A
25,CAPH2-AID-G2-S5-R7-spNA,CAPH2,G2,S5,False,True,True
26,CAPH2-AID-10min-S5-R5-spNA,CAPH2,10min,S5,False,True,True
27,CAPH2-AID-15min-S5-R6-spNA,CAPH2,15min,S5,False,True,True
28,CAPH2-AID-30min-S5-R9-spNA,CAPH2,30min,S5,False,True,True
29,CAPH2-AID-60min-S5-R3-spNA,CAPH2,60min,S5,False,True,True


,lib_name,condition,time,replicate,S,N/M,A
30,CAPH2-AID-G2-S6-R8-spNa,CAPH2,G2,S6,False,True,False
31,CAPH2-AID-10min-S6-R6-spNa,CAPH2,10min,S6,False,True,False
32,CAPH2-AID-15min-S6-R7-spNa,CAPH2,15min,S6,False,True,False
33,CAPH2-AID-30min-S6-R10-spNa,CAPH2,30min,S6,False,True,False
34,CAPH2-AID-60min-S6-R3-spNa,CAPH2,60min,S6,False,True,False


...

,lib_name,condition,time,replicate,S,N/M,A
222,WAPL-AID-G2-S4-R2-spna,WAPL,G2,S4,False,False,False
223,WAPL-AID-2.5min-S4-R2-spna,WAPL,2.5min,S4,False,False,False
224,WAPL-AID-5min-S4-R2-spna,WAPL,5min,S4,False,False,False
225,WAPL-AID-7.5min-S4-R2-spna,WAPL,7.5min,S4,False,False,False
226,WAPL-AID-10min-S4-R2-spna,WAPL,10min,S4,False,False,False
227,WAPL-AID-15min-S4-R2-spna,WAPL,15min,S4,False,False,False
228,WAPL-AID-30min-S4-R2-spna,WAPL,30min,S4,False,False,False


,lib_name,condition,time,replicate,S,N/M,A
229,WAPL-AID-G2-S5-R3-spnA,WAPL,G2,S5,False,False,True
230,WAPL-AID-2.5min-S5-R3-spnA,WAPL,2.5min,S5,False,False,True
231,WAPL-AID-5min-S5-R3-spnA,WAPL,5min,S5,False,False,True
232,WAPL-AID-7.5min-S5-R3-spnA,WAPL,7.5min,S5,False,False,True
233,WAPL-AID-10min-S5-R3-spnA,WAPL,10min,S5,False,False,True
234,WAPL-AID-15min-S5-R3-spnA,WAPL,15min,S5,False,False,True
235,WAPL-AID-30min-S5-R3-spnA,WAPL,30min,S5,False,False,True


,lib_name,condition,time,replicate,S,N/M,A
236,WT-G2-S4-R5-spna,WT,G2,S4,False,False,False
237,WT-2.5min-S4-R2-spna,WT,2.5min,S4,False,False,False
238,WT-5min-S4-R3-spna,WT,5min,S4,False,False,False
239,WT-7.5min-S4-R2-spna,WT,7.5min,S4,False,False,False
240,WT-10min-S4-R3-spna,WT,10min,S4,False,False,False
241,WT-15min-S4-R3-spna,WT,15min,S4,False,False,False


,lib_name,condition,time,replicate,S,N/M,A
242,WT-G2-S5-R6-spna,WT,G2,S5,False,False,False
243,WT-G2-S5-R7-spnA,WT,G2,S5,False,False,True
244,WT-G2-S5-R8-spna,WT,G2,S5,False,False,False
245,WT-2.5min-S5-R3-spna,WT,2.5min,S5,False,False,False
246,WT-5min-S5-R4-spna,WT,5min,S5,False,False,False
247,WT-7.5min-S5-R3-spna,WT,7.5min,S5,False,False,False
248,WT-10min-S5-R4-spna,WT,10min,S5,False,False,False
249,WT-15min-S5-R4-spna,WT,15min,S5,False,False,False
250,WT-30min-S5-R5-spna,WT,30min,S5,False,False,False
251,WT-60min-S5-R4-spMa,WT,60min,S5,False,True,False


,lib_name,condition,time,replicate,S,N/M,A
253,WT-G2-S6-R9-spna,WT,G2,S6,False,False,False
254,WT-30min-S6-R6-spNa,WT,30min,S6,False,True,False


### Assigning preferred repicates for downstream analysis

In [4]:
df.loc[:, 'preferred'] = False

caph = (df['condition'] == 'CAPH') & (df['replicate'] == 'S3') 
df.loc[caph, 'preferred'] = True

caph2 = (df['condition'] == 'CAPH2') & (df['replicate'] == 'S3') 
df.loc[caph2, 'preferred'] = True

ctcf = (df['condition'] == 'CTCF') 
df.loc[ctcf, 'preferred'] = True

kif4 = (df['condition'] == 'KIF4') 
df.loc[kif4, 'preferred'] = True

smc2 = (df['condition'] == 'SMC2') & (df['replicate'] == 'S4') 
df.loc[smc2, 'preferred'] = True

smc2_wapl = (df['condition'] == 'SMC2-WAPL') & (df['replicate'] == 'S2') 
df.loc[smc2_wapl, 'preferred'] = True

smc3 = (df['condition'] == 'SMC3') 
smc3 = smc3 & (((df['replicate'] == 'S1') | (df['replicate'] == 'S2')) | (df['replicate'] == 'S4')) 
df.loc[smc3, 'preferred'] = True

smc3_caph = (df['condition'] == 'SMC3-CAPH') & ((df['replicate'] == 'S3') | (df['replicate'] == 'S2'))
df.loc[smc3_caph, 'preferred'] = True

smc3_caph2 = (df['condition'] == 'SMC3-CAPH2') & ((df['replicate'] == 'S3') | (df['replicate'] == 'S2'))
df.loc[smc3_caph2, 'preferred'] = True

smc3_ctcf = (df['condition'] == 'SMC3-CTCF') 
df.loc[smc3_ctcf, 'preferred'] = True

smc3_kif4 = (df['condition'] == 'SMC3-KIF4') 
df.loc[smc3_kif4, 'preferred'] = True

smc3_smc2 = (df['condition'] == 'SMC3-SMC2') 
df.loc[smc3_smc2, 'preferred'] = True

smc5 = (df['condition'] == 'SMC5') 
df.loc[smc5, 'preferred'] = True

wapl = (df['condition'] == 'WAPL') & ((df['replicate'] == 'S5') | (df['replicate'] == 'S4'))
df.loc[wapl, 'preferred'] = True

wt = (df['condition'] == 'WT')
m1 = wt & ((df['replicate'] == 'S4') & (df['time'] < '30min'))
m2 = wt & ((df['replicate'] == 'S5') & (df['time'] == '30min'))
wt = m1 | m2
df.loc[wt, 'preferred'] = True

for _, group in df.groupby(['condition','replicate']):
    display(group)

,lib_name,condition,time,replicate,S,N/M,A,preferred
0,CAPH-AID-G2-S3-R5-spnA,CAPH,G2,S3,False,False,True,True
1,CAPH-AID-2.5min-S3-R3-spnA,CAPH,2.5min,S3,False,False,True,True
2,CAPH-AID-7.5min-S3-R3-spnA,CAPH,5min,S3,False,False,True,True
3,CAPH-AID-5min-S3-R3-spnA,CAPH,7.5min,S3,False,False,True,True
4,CAPH-AID-10min-S3-R3-spnA,CAPH,10min,S3,False,False,True,True
5,CAPH-AID-15min-S3-R3-spnA,CAPH,15min,S3,False,False,True,True
6,CAPH-AID-30min-S3-R3-spnA,CAPH,30min,S3,False,False,True,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
7,CAPH2-AID-G2-S3-R5-spnA,CAPH2,G2,S3,False,False,True,True
8,CAPH2-AID-2.5min-S3-R3-spnA,CAPH2,2.5min,S3,False,False,True,True
9,CAPH2-AID-5min-S3-R3-spnA,CAPH2,5min,S3,False,False,True,True
10,CAPH2-AID-7.5min-S3-R3-spnA,CAPH2,7.5min,S3,False,False,True,True
11,CAPH2-AID-10min-S3-R3-spnA,CAPH2,10min,S3,False,False,True,True
12,CAPH2-AID-15min-S3-R3-spnA,CAPH2,15min,S3,False,False,True,True
13,CAPH2-AID-30min-S3-R3-spnA,CAPH2,30min,S3,False,False,True,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
14,CAPH2-AID-G2-S4-R6-spnA,CAPH2,G2,S4,False,False,True,False
15,CAPH2-AID-5min-S4-R4-spnA,CAPH2,5min,S4,False,False,True,False
16,CAPH2-AID-10min-S4-R4-spnA,CAPH2,10min,S4,False,False,True,False
17,CAPH2-AID-15min-S4-R4-spnA,CAPH2,15min,S4,False,False,True,False
18,CAPH2-AID-15min-S4-R5-spNA,CAPH2,15min,S4,False,True,True,False
19,CAPH2-AID-15min-S4-R6-spNA,CAPH2,15min,S4,False,True,True,False
20,CAPH2-AID-30min-S4-R4-spNa,CAPH2,30min,S4,False,True,False,False
21,CAPH2-AID-30min-S4-R4-spnA,CAPH2,30min,S4,False,False,True,False
22,CAPH2-AID-30min-S4-R5-spNA,CAPH2,30min,S4,False,True,True,False
23,CAPH2-AID-30min-S4-R6-spNA,CAPH2,30min,S4,False,True,True,False


,lib_name,condition,time,replicate,S,N/M,A,preferred
25,CAPH2-AID-G2-S5-R7-spNA,CAPH2,G2,S5,False,True,True,False
26,CAPH2-AID-10min-S5-R5-spNA,CAPH2,10min,S5,False,True,True,False
27,CAPH2-AID-15min-S5-R6-spNA,CAPH2,15min,S5,False,True,True,False
28,CAPH2-AID-30min-S5-R9-spNA,CAPH2,30min,S5,False,True,True,False
29,CAPH2-AID-60min-S5-R3-spNA,CAPH2,60min,S5,False,True,True,False


,lib_name,condition,time,replicate,S,N/M,A,preferred
30,CAPH2-AID-G2-S6-R8-spNa,CAPH2,G2,S6,False,True,False,False
31,CAPH2-AID-10min-S6-R6-spNa,CAPH2,10min,S6,False,True,False,False
32,CAPH2-AID-15min-S6-R7-spNa,CAPH2,15min,S6,False,True,False,False
33,CAPH2-AID-30min-S6-R10-spNa,CAPH2,30min,S6,False,True,False,False
34,CAPH2-AID-60min-S6-R3-spNa,CAPH2,60min,S6,False,True,False,False


...

,lib_name,condition,time,replicate,S,N/M,A,preferred
222,WAPL-AID-G2-S4-R2-spna,WAPL,G2,S4,False,False,False,True
223,WAPL-AID-2.5min-S4-R2-spna,WAPL,2.5min,S4,False,False,False,True
224,WAPL-AID-5min-S4-R2-spna,WAPL,5min,S4,False,False,False,True
225,WAPL-AID-7.5min-S4-R2-spna,WAPL,7.5min,S4,False,False,False,True
226,WAPL-AID-10min-S4-R2-spna,WAPL,10min,S4,False,False,False,True
227,WAPL-AID-15min-S4-R2-spna,WAPL,15min,S4,False,False,False,True
228,WAPL-AID-30min-S4-R2-spna,WAPL,30min,S4,False,False,False,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
229,WAPL-AID-G2-S5-R3-spnA,WAPL,G2,S5,False,False,True,True
230,WAPL-AID-2.5min-S5-R3-spnA,WAPL,2.5min,S5,False,False,True,True
231,WAPL-AID-5min-S5-R3-spnA,WAPL,5min,S5,False,False,True,True
232,WAPL-AID-7.5min-S5-R3-spnA,WAPL,7.5min,S5,False,False,True,True
233,WAPL-AID-10min-S5-R3-spnA,WAPL,10min,S5,False,False,True,True
234,WAPL-AID-15min-S5-R3-spnA,WAPL,15min,S5,False,False,True,True
235,WAPL-AID-30min-S5-R3-spnA,WAPL,30min,S5,False,False,True,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
236,WT-G2-S4-R5-spna,WT,G2,S4,False,False,False,True
237,WT-2.5min-S4-R2-spna,WT,2.5min,S4,False,False,False,True
238,WT-5min-S4-R3-spna,WT,5min,S4,False,False,False,True
239,WT-7.5min-S4-R2-spna,WT,7.5min,S4,False,False,False,True
240,WT-10min-S4-R3-spna,WT,10min,S4,False,False,False,True
241,WT-15min-S4-R3-spna,WT,15min,S4,False,False,False,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
242,WT-G2-S5-R6-spna,WT,G2,S5,False,False,False,False
243,WT-G2-S5-R7-spnA,WT,G2,S5,False,False,True,False
244,WT-G2-S5-R8-spna,WT,G2,S5,False,False,False,False
245,WT-2.5min-S5-R3-spna,WT,2.5min,S5,False,False,False,False
246,WT-5min-S5-R4-spna,WT,5min,S5,False,False,False,False
247,WT-7.5min-S5-R3-spna,WT,7.5min,S5,False,False,False,False
248,WT-10min-S5-R4-spna,WT,10min,S5,False,False,False,False
249,WT-15min-S5-R4-spna,WT,15min,S5,False,False,False,False
250,WT-30min-S5-R5-spna,WT,30min,S5,False,False,False,True
251,WT-60min-S5-R4-spMa,WT,60min,S5,False,True,False,False


,lib_name,condition,time,replicate,S,N/M,A,preferred
253,WT-G2-S6-R9-spna,WT,G2,S6,False,False,False,False
254,WT-30min-S6-R6-spNa,WT,30min,S6,False,True,False,False


### Persistent database creation  using the shelve libarary

In [5]:
db_path = '/home/sameer/mitosis/galGal7/metadata/galGal7_info'
with shelve.open(db_path, flag='c') as db:
    db['metadata'] = df

# Testing the mitosis_manager script

### Initial importation of database

In [2]:
imp.reload(mm)
db = mm.Dataset()
table = db.get_tables()
for cond, group in table.groupby(['condition', 'replicate']):
    display(group)

,lib_name,condition,time,replicate,S,N/M,A,preferred
0,CAPH-AID-G2-S3-R5-spnA,CAPH,G2,S3,False,False,True,True
1,CAPH-AID-2.5min-S3-R3-spnA,CAPH,2.5min,S3,False,False,True,True
2,CAPH-AID-7.5min-S3-R3-spnA,CAPH,5min,S3,False,False,True,True
3,CAPH-AID-5min-S3-R3-spnA,CAPH,7.5min,S3,False,False,True,True
4,CAPH-AID-10min-S3-R3-spnA,CAPH,10min,S3,False,False,True,True
5,CAPH-AID-15min-S3-R3-spnA,CAPH,15min,S3,False,False,True,True
6,CAPH-AID-30min-S3-R3-spnA,CAPH,30min,S3,False,False,True,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
7,CAPH2-AID-G2-S3-R5-spnA,CAPH2,G2,S3,False,False,True,True
8,CAPH2-AID-2.5min-S3-R3-spnA,CAPH2,2.5min,S3,False,False,True,True
9,CAPH2-AID-5min-S3-R3-spnA,CAPH2,5min,S3,False,False,True,True
10,CAPH2-AID-7.5min-S3-R3-spnA,CAPH2,7.5min,S3,False,False,True,True
11,CAPH2-AID-10min-S3-R3-spnA,CAPH2,10min,S3,False,False,True,True
12,CAPH2-AID-15min-S3-R3-spnA,CAPH2,15min,S3,False,False,True,True
13,CAPH2-AID-30min-S3-R3-spnA,CAPH2,30min,S3,False,False,True,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
14,CAPH2-AID-G2-S4-R6-spnA,CAPH2,G2,S4,False,False,True,False
15,CAPH2-AID-5min-S4-R4-spnA,CAPH2,5min,S4,False,False,True,False
16,CAPH2-AID-10min-S4-R4-spnA,CAPH2,10min,S4,False,False,True,False
17,CAPH2-AID-15min-S4-R4-spnA,CAPH2,15min,S4,False,False,True,False
18,CAPH2-AID-15min-S4-R5-spNA,CAPH2,15min,S4,False,True,True,False
19,CAPH2-AID-15min-S4-R6-spNA,CAPH2,15min,S4,False,True,True,False
20,CAPH2-AID-30min-S4-R4-spNa,CAPH2,30min,S4,False,True,False,False
21,CAPH2-AID-30min-S4-R4-spnA,CAPH2,30min,S4,False,False,True,False
22,CAPH2-AID-30min-S4-R5-spNA,CAPH2,30min,S4,False,True,True,False
23,CAPH2-AID-30min-S4-R6-spNA,CAPH2,30min,S4,False,True,True,False


,lib_name,condition,time,replicate,S,N/M,A,preferred
25,CAPH2-AID-G2-S5-R7-spNA,CAPH2,G2,S5,False,True,True,False
26,CAPH2-AID-10min-S5-R5-spNA,CAPH2,10min,S5,False,True,True,False
27,CAPH2-AID-15min-S5-R6-spNA,CAPH2,15min,S5,False,True,True,False
28,CAPH2-AID-30min-S5-R9-spNA,CAPH2,30min,S5,False,True,True,False
29,CAPH2-AID-60min-S5-R3-spNA,CAPH2,60min,S5,False,True,True,False


,lib_name,condition,time,replicate,S,N/M,A,preferred
30,CAPH2-AID-G2-S6-R8-spNa,CAPH2,G2,S6,False,True,False,False
31,CAPH2-AID-10min-S6-R6-spNa,CAPH2,10min,S6,False,True,False,False
32,CAPH2-AID-15min-S6-R7-spNa,CAPH2,15min,S6,False,True,False,False
33,CAPH2-AID-30min-S6-R10-spNa,CAPH2,30min,S6,False,True,False,False
34,CAPH2-AID-60min-S6-R3-spNa,CAPH2,60min,S6,False,True,False,False


...

,lib_name,condition,time,replicate,S,N/M,A,preferred
222,WAPL-AID-G2-S4-R2-spna,WAPL,G2,S4,False,False,False,True
223,WAPL-AID-2.5min-S4-R2-spna,WAPL,2.5min,S4,False,False,False,True
224,WAPL-AID-5min-S4-R2-spna,WAPL,5min,S4,False,False,False,True
225,WAPL-AID-7.5min-S4-R2-spna,WAPL,7.5min,S4,False,False,False,True
226,WAPL-AID-10min-S4-R2-spna,WAPL,10min,S4,False,False,False,True
227,WAPL-AID-15min-S4-R2-spna,WAPL,15min,S4,False,False,False,True
228,WAPL-AID-30min-S4-R2-spna,WAPL,30min,S4,False,False,False,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
229,WAPL-AID-G2-S5-R3-spnA,WAPL,G2,S5,False,False,True,True
230,WAPL-AID-2.5min-S5-R3-spnA,WAPL,2.5min,S5,False,False,True,True
231,WAPL-AID-5min-S5-R3-spnA,WAPL,5min,S5,False,False,True,True
232,WAPL-AID-7.5min-S5-R3-spnA,WAPL,7.5min,S5,False,False,True,True
233,WAPL-AID-10min-S5-R3-spnA,WAPL,10min,S5,False,False,True,True
234,WAPL-AID-15min-S5-R3-spnA,WAPL,15min,S5,False,False,True,True
235,WAPL-AID-30min-S5-R3-spnA,WAPL,30min,S5,False,False,True,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
236,WT-G2-S4-R5-spna,WT,G2,S4,False,False,False,True
237,WT-2.5min-S4-R2-spna,WT,2.5min,S4,False,False,False,True
238,WT-5min-S4-R3-spna,WT,5min,S4,False,False,False,True
239,WT-7.5min-S4-R2-spna,WT,7.5min,S4,False,False,False,True
240,WT-10min-S4-R3-spna,WT,10min,S4,False,False,False,True
241,WT-15min-S4-R3-spna,WT,15min,S4,False,False,False,True


,lib_name,condition,time,replicate,S,N/M,A,preferred
242,WT-G2-S5-R6-spna,WT,G2,S5,False,False,False,False
243,WT-G2-S5-R7-spnA,WT,G2,S5,False,False,True,False
244,WT-G2-S5-R8-spna,WT,G2,S5,False,False,False,False
245,WT-2.5min-S5-R3-spna,WT,2.5min,S5,False,False,False,False
246,WT-5min-S5-R4-spna,WT,5min,S5,False,False,False,False
247,WT-7.5min-S5-R3-spna,WT,7.5min,S5,False,False,False,False
248,WT-10min-S5-R4-spna,WT,10min,S5,False,False,False,False
249,WT-15min-S5-R4-spna,WT,15min,S5,False,False,False,False
250,WT-30min-S5-R5-spna,WT,30min,S5,False,False,False,True
251,WT-60min-S5-R4-spMa,WT,60min,S5,False,True,False,False


,lib_name,condition,time,replicate,S,N/M,A,preferred
253,WT-G2-S6-R9-spna,WT,G2,S6,False,False,False,False
254,WT-30min-S6-R6-spNa,WT,30min,S6,False,True,False,False


### Adding a column to the database 

In [5]:
dummy_df = table[['lib_name','A']].copy(deep=True)
dummy_df.rename(columns={'A':'auxin'}, inplace=True)
dummy_df.head()

,lib_name,auxin
0,CAPH-AID-G2-S3-R5-spnA,True
1,CAPH-AID-2.5min-S3-R3-spnA,True
2,CAPH-AID-7.5min-S3-R3-spnA,True
3,CAPH-AID-5min-S3-R3-spnA,True
4,CAPH-AID-10min-S3-R3-spnA,True


In [6]:
db.add_table('dummy', dummy_df)

In [10]:
db2 = mm.Dataset()
table = db2.get_tables('dummy')
table.head()

,lib_name,condition,time,replicate,S,N/M,A,preferred,auxin
0,CAPH-AID-G2-S3-R5-spnA,CAPH,G2,S3,False,False,True,True,True
1,CAPH-AID-2.5min-S3-R3-spnA,CAPH,2.5min,S3,False,False,True,True,True
2,CAPH-AID-7.5min-S3-R3-spnA,CAPH,5min,S3,False,False,True,True,True
3,CAPH-AID-5min-S3-R3-spnA,CAPH,7.5min,S3,False,False,True,True,True
4,CAPH-AID-10min-S3-R3-spnA,CAPH,10min,S3,False,False,True,True,True


### Modifying an existing table in the database

In [12]:
modified_dummy = dummy_df.copy(deep=True)
modified_dummy['auxin'] = modified_dummy['auxin'].apply(lambda x: 1 if x else 0)
modified_dummy.head()

,lib_name,auxin
0,CAPH-AID-G2-S3-R5-spnA,1
1,CAPH-AID-2.5min-S3-R3-spnA,1
2,CAPH-AID-7.5min-S3-R3-spnA,1
3,CAPH-AID-5min-S3-R3-spnA,1
4,CAPH-AID-10min-S3-R3-spnA,1


In [15]:
db.modify_table('dummy', modified_dummy)

In [14]:
db2 = mm.Dataset()
table = db2.get_tables('dummy')
table.head()

,lib_name,condition,time,replicate,S,N/M,A,preferred,auxin
0,CAPH-AID-G2-S3-R5-spnA,CAPH,G2,S3,False,False,True,True,1
1,CAPH-AID-2.5min-S3-R3-spnA,CAPH,2.5min,S3,False,False,True,True,1
2,CAPH-AID-7.5min-S3-R3-spnA,CAPH,5min,S3,False,False,True,True,1
3,CAPH-AID-5min-S3-R3-spnA,CAPH,7.5min,S3,False,False,True,True,1
4,CAPH-AID-10min-S3-R3-spnA,CAPH,10min,S3,False,False,True,True,1


### Removing table from dataset

In [ ]:
db.remove_table('dummy')

In [ ]:
db2 = mm.Dataset()
table = db2.get_tables('dummy')

### FIltering data

In [ ]:
imp.reload(mm)
db = mm.Dataset()
table = db.get_tables()
table
display(mm.filter_data(table, {'condition':'SMC3', 'A':'Aa', 'preferred':True}))
# display(mm.filter_data(table, {'condition':['SMC3','SMC3-CAPH', 'SMC3-CAPH2','SMC3-SMC2'],
#                                'aux':True, 'preferred':True}))

### Fetching coolers

In [ ]:
mm.get_coolers(table, 1000)

In [ ]:
mm.get_coolers(table, 1000, filepath=True)

### Fetching pairs

In [ ]:
imp.reload(mm)
mm.get_pairs(table)

# Colors

In [ ]:
condition_colors = {'WT':'#267326',
                    
                    'SMC3 +':'#0066cc',
                    'SMC3 -':'#267326',
                    
                    'SMC3 --':'#267326',
                    'SMC3 +-':'#53c653',
                    'SMC3 -+':'#0066cc',
                    'SMC3 ++':'#4da6ff',
                    
                    'SMC3-SMC2':'#b30000',
                    'SMC2':'#ff8080',
                    
                    'SMC3-CAPH':'#7b3294',
                    'CAPH':'#bc7ad1',
                    
                    'SMC3-CAPH2':'#b14e02',
                    'CAPH2':'#fd8c35',
                     }



In [ ]:
condition_colors = {'WT':'#b14e02', #35
                    
                    'SMC3 +':'#0073e6', #45
                    'SMC3 -':'#b14e02', #35
                    
                    'SMC3 --':'#b14e02', #35
                    'SMC3 +-':'#fd8c35', #60
                    'SMC3 -+':'#0073e6', #45
                    'SMC3 ++':'#66b3ff', #70
                    
                    'SMC3-SMC2':'#913bab', #45
                    'SMC2':'#c68ed7', #70
                    
                    'SMC3-CAPH':'#e60000', #45 
                    'CAPH':'#ff6666', #70
                    
                    'SMC3-CAPH2':'#39ac39', #45
                    'CAPH2':'#8cd98c', #70
                     }



In [ ]:
%matplotlib notebook

In [ ]:
n = len(condition_colors)

fig, ax = plt.subplots(figsize=(4,0.25*n), constrained_layout=True)

for i, key in enumerate(condition_colors.keys()):
    ax.axhline(i,color=condition_colors[key], linewidth=10)
    
ax.set_yticks(np.arange(n))
ax.set_xticks([])
_ = ax.set_yticklabels(list(condition_colors.keys()))

In [ ]:
fig = plt.figure(figsize=(4*2,0.3*9))
gs = GridSpec(nrows=9, ncols=2)

lw = 20


keys = ['WT','SMC3 -','SMC3 --','SMC3 +-']
ax = fig.add_subplot(gs[0:5,0])

ax.set_title('Identical Hi-C in G2', fontweight='bold')
for i, k in enumerate(keys):
    ax.axhline(i,color=condition_colors[k], linewidth=lw)
    ax.text(1, i-0.15, condition_colors[k])

ax.set_ylim([-0.25,3+0.25])
ax.set_yticks(np.arange(4))
ax.set_xticks([])
_ = ax.set_yticklabels(keys, fontweight='bold')

keys = ['SMC3 +','SMC3 -+','SMC3 ++']
ax = fig.add_subplot(gs[5:,0])

for i, k in enumerate(keys):
    ax.axhline(i,color=condition_colors[k], linewidth=lw)
    ax.text(1, i-0.15, condition_colors[k])
    
ax.set_ylim([-0.25,2+0.25])
ax.set_yticks(np.arange(3))
ax.set_xticks([])
_ = ax.set_yticklabels(keys, fontweight='bold')

keys = ['SMC2','SMC3-SMC2']
ax = fig.add_subplot(gs[0:3,1])

ax.set_title('Double v Single', fontweight='bold')
for i, k in enumerate(keys):
    ax.axhline(i,color=condition_colors[k], linewidth=lw)
    ax.text(1, i-0.15, condition_colors[k])
    
ax.set_ylim([-0.25,1+0.25])
ax.set_yticks(np.arange(2))
ax.set_xticks([])
_ = ax.set_yticklabels(keys, fontweight='bold')

keys = ['CAPH','SMC3-CAPH']
ax = fig.add_subplot(gs[3:6,1])

for i, k in enumerate(keys):
    ax.axhline(i,color=condition_colors[k], linewidth=lw)
    ax.text(1, i-0.15, condition_colors[k])
    
ax.set_ylim([-0.25,1+0.25])
ax.set_yticks(np.arange(2))
ax.set_xticks([])
_ = ax.set_yticklabels(keys, fontweight='bold')

keys = ['CAPH2','SMC3-CAPH2']
ax = fig.add_subplot(gs[6:,1])

for i, k in enumerate(keys):
    ax.axhline(i,color=condition_colors[k], linewidth=lw)
    ax.text(1, i-0.15, condition_colors[k])
    
ax.set_ylim([-0.25,1+0.25])
ax.set_yticks(np.arange(2))
ax.set_xticks([])
_ = ax.set_yticklabels(keys, fontweight='bold')

fig.tight_layout()
# fig.savefig('/home/sameer/Dropbox (MIT)/Research/mitosis/manuscript/scaling_colors.png',format='png')